In [ ]:
# stages:
# 1. Load a large set of geotiffs as xarray datasets
# 2. Select a few 32x32 pixel chips from each xarray
# 3. Generate embeddings for each chip
# 4. Create two lists. One containing the geometries of the chips and the other containing the embeddings

In [ ]:
from datetime import datetime
import geopandas as gpd
import glob
import matplotlib.pyplot as plt
from multiprocessing import get_context
import numpy as np
import random
import rioxarray
from shapely.geometry import box, Point
from scipy import spatial
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers
import ipyleaflet as ipyl
import ipywidgets as ipyw

from gee.constants import S2_BANDS, REDUCERS

In [ ]:
def load_xarray(file_name, chips_per_tile=2, patch_size=32):
    band_names = [f"{band}_{reducer}" for band in S2_BANDS for reducer in REDUCERS]
    data = rioxarray.open_rasterio(file_name)
    data['band'] = band_names
    data['time'] = datetime(2020, 1, 1)
    data = data.transpose('y', 'x', 'band')
    bboxes = []
    mean_chips = []
    std_chips = []
    for i in range(chips_per_tile):
        chip = extract_subset(data, patch_size)
        # get the bounding box of the dataset
        bbox = chip.rio.bounds()
        mean_chip = get_reducer(chip, 'mean').to_numpy()
        std_chip = get_reducer(chip, 'stdDev').to_numpy()
        bboxes.append(bbox)
        mean_chips.append(mean_chip)
        std_chips.append(std_chip)
    return mean_chips, std_chips, bbox

def load_xarrays(path, max_files=-1, num_workers=12):
    """Load xarrays from a directory"""
    
    # get all the files in the directory
    files = glob.glob(path + '*.tif')
    # select a random subset of files
    if max_files > 0:
        files = random.sample(files, max_files)
    # load the files into xarrays using multiprocessing
    args = [(f, 1, 32) for f in files]
    with get_context('fork').Pool(num_workers) as pool:
        xarrays = pool.starmap(load_xarray, args)
    return xarrays

# take a random square subset of the data
def extract_subset(xarr, size):
    # extract a random patch of height and width size
    x = random.randint(0, xarr.shape[1] - size)
    y = random.randint(0, xarr.shape[0] - size)
    return xarr[y:y+size, x:x+size, :]

def get_rgb(xarr):
    return xarr.sel(band=['B4_mean', 'B3_mean', 'B2_mean'])

def get_reducer(xarr, reducer):
    return xarr.sel(band=[f"{band}_{reducer}" for band in S2_BANDS])

def unit_norm(chips):
    # Means and standard deviations used for normalization
    # The mean value bands and the standard deviation bands are normalized separately
    # If this works, move these to a constants.py file
    MEAN_MEAN = [336.802, 531.024, 468.612, 886.337, 1917.027, 2276.156, 2436.756, 2546.673, 1766.056, 983.634, 0.648, -0.614]
    STD_MEAN = [131.763, 169.545, 219.578, 245.352, 317.504, 363.269, 421.290, 403.054, 484.412, 361.534, 0.122, 0.100]
    MEAN_STD = [120.957, 141.433, 162.636, 176.621, 567.319, 708.051, 676.749, 709.860, 291.253, 244.241, 0.120, 0.076]
    STD_STD = [51.520, 58.440, 85.941, 71.694, 177.075, 227.558, 238.055, 224.945, 126.288, 127.786, 0.047, 0.029]
    # normalize the mean and stdDev chips
    mean_chips = chips[:,:,:,:12]
    std_chips = chips[:,:,:,12:]
    mean_chips_norm = [np.clip((mean_chip - np.tile(MEAN_MEAN, (32, 32, 1))) / np.tile(STD_MEAN, (32, 32, 1)), -3, 3) for mean_chip in mean_chips]    
    std_chips_norm = [np.clip((std_chip - np.tile(MEAN_STD, (32, 32, 1))) / np.tile(STD_STD, (32, 32, 1)), -3, 3) for std_chip in std_chips]
    # return chips in array of shape (num_chips, 32, 32, 24)
    return np.concatenate((mean_chips_norm, std_chips_norm), axis=3)

In [ ]:
data_directory = '../../alabama/'
files = glob.glob(data_directory + '*.tif')
band_names = [f"{band}_{reducer}" for band in S2_BANDS for reducer in REDUCERS]
chips_per_tile = 1
patch_size = 32

mean_chips = []
std_chips = []
bboxes = []

num_files = 5000
num_workers = 12
input_data = load_xarrays(data_directory, num_files, num_workers)
# concatenate mean and std chips together into shape 32x32x24
chips = np.concatenate(([x[0][0] for x in input_data], [x[1][0] for x in input_data]), axis=-1)
# normalize
norm_chips = unit_norm(chips)

# create a geodataframe of bounding boxes
bboxes = [x[2] for x in input_data]
geometries = gpd.GeoDataFrame(geometry=[box(*bbox) for bbox in bboxes])
# convert from Alabama UTM to WGS84. 
# Not guaranteed to be accurate if embeddings are from multiple UTM zones
geometries = geometries.set_crs(epsg=32616).to_crs(epsg=4326)

In [ ]:
# Import the model
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = keras.backend.shape(z_mean)[0]
        dim = keras.backend.shape(z_mean)[1]
        epsilon = keras.backend.random_normal(shape=(batch, dim))
        return z_mean + keras.backend.exp(0.5 * z_log_var) * epsilon

model_path = '../models/vae_encoder_12channel_2022-11-08.h5'
model = load_model(model_path, custom_objects={'Sampling': Sampling})
decoder_path = '../models/vae_decoder_12channel_2022-11-08.h5'
decoder = load_model(decoder_path, custom_objects={'Sampling': Sampling})

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Autoencoder(nn.Module):
    #This is the model where we create the set of learned filters
    def __init__(self, num_filters=64):
        super(Autoencoder, self).__init__()
        self.conv1 = nn.Conv2d(24, num_filters, 3, stride=1, padding="same")
        self.conv2 = nn.Conv2d(num_filters, num_filters, 3, stride=1, padding="same")
        self.conv3 = nn.Conv2d(num_filters, num_filters, 3, stride=1, padding="same")
        self.conv4 = nn.Conv2d(num_filters, num_filters, 3, stride=1, padding="same")
        self.conv5 = nn.Conv2d(num_filters, num_filters, 3, stride=1, padding="same")
        self.conv6 = nn.Conv2d(num_filters, 24, 3, stride=1, padding="same")

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = F.max_pool2d(x + F.leaky_relu(self.conv2(x)), 2)
        x = F.max_pool2d(x + F.leaky_relu(self.conv3(x)), 2)
        x = F.interpolate(x + F.leaky_relu(self.conv4(x)), x.shape[-1]*2)
        x = F.interpolate(x + F.leaky_relu(self.conv5(x)), x.shape[-1]*2)
        x = F.leaky_relu(self.conv6(x))
        
        return x
    
    def encode(self, x, flatten=True):
        x = F.leaky_relu(self.conv1(x))
        x = F.max_pool2d(x + F.leaky_relu(self.conv2(x)), 2)
        x = F.max_pool2d(x + F.leaky_relu(self.conv3(x)), 2)
        x = x + F.leaky_relu(self.conv4(x))
        
        if flatten:
            return torch.flatten(x, start_dim = 1)
        else:
            return x

device = "cpu" #change this to "cuda" for GPU or "mps" for Apple M1 chip

autoencoder = torch.load("/Users/ckruse/Downloads/residual_autoencoder.pt", map_location=device)

X = torch.tensor(norm_chips, dtype=torch.float32, device=device) #convert a small batch to a torch tensor
X = X.permute((0, 3, 1, 2)) #PyTorch is channels first, so we do a quick permute here

preds = autoencoder.encode(X).detach().cpu().numpy() #get the encodings as a numpy array

print(preds.shape)

In [ ]:
model = keras.models.load_model('/Users/ckruse/Downloads/test_general_model.h5')

In [ ]:
model.summary()

In [ ]:
# get the outputs of the model after the convolutional layers
conv_model = keras.Model(inputs=model.input, outputs=model.get_layer('flatten_2').output)
conv_model.summary()

In [ ]:
chips.shape

In [ ]:
preds = conv_model.predict(np.clip(chips[:,:,:,1:4] / 4000, 0, 1))

In [ ]:
plt.hist(preds.flatten(), bins=100);

In [ ]:
plt.figure(figsize=(8,5),facecolor=(1,1,1))
plt.hist(preds[0:100].flatten(), bins=100)
plt.xlabel("Encoding value")
plt.ylabel("Count")
plt.show()

In [ ]:
preds = model.predict(norm_chips[:,:,:,:12])

In [ ]:
preds.shape

In [ ]:
# create a leaflet map with all geometries
# define map parameters
map_kwargs = dict()

map_kwargs['zoom'] = 4
map_kwargs['zoom_control'] = False
map_kwargs['attribution_control'] = True
map_kwargs['scroll_wheel_zoom'] = True
map_kwargs["no_wrap"] = True
map_kwargs["prefer_canvas"] = True
controls = ipyl.ScaleControl(options=["update_when_idle"])

# build mapbox basemap
MAPBOX_TOKEN = 'pk.eyJ1IjoiZWFydGhyaXNlLWRldiIsImEiOiJjazFrMmRwM3Mwa2xkM2VxN3c5YnIxMXFiIn0.AzF5Q7NFuOAGZWkOr9n8CA'
MAPBOX_URL = 'https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v11/tiles/512/{z}/{x}/{y}@2x?access_token='
mapbox_url = MAPBOX_URL + MAPBOX_TOKEN
mapbox_layer = ipyl.TileLayer(url=mapbox_url, no_wrap=True, controls=controls, name="basemap",
                            attribution="© <a href='https://www.mapbox.com/about/maps/'>Mapbox</a> © <a href='http://www.openstreetmap.org/copyright'>OpenStreetMap</a> <strong><a>")
# build map
m = ipyl.Map(basemap=mapbox_layer, **map_kwargs)

# set layout height
m.layout = ipyw.Layout(height='800px')
# add geometries to map
m.add_layer(ipyl.GeoJSON(data=geometries.__geo_interface__, style={'color': 'red'}))
# set the map center to the center of the first geometry
# get the centroid of the geodataframe bounding box
centroid = geometries.geometry[0].centroid.coords[0]
# get the lat and lon of the centroid
lat = centroid[1]
lon = centroid[0]
m.center = (lat, lon)
# set the appropriate zoom
m.zoom = 14
clicked_point = None
# get the coordinates of a clicked point
def get_coords(**kwargs):
    # look up the geometry using a spatial index that contains contains the clicked point and return the index
    if kwargs['type'] == 'click':
        point = Point(
            kwargs['coordinates'][1],
            kwargs['coordinates'][0]
        )
        # get the index of the clicked point in the geodataframe
        search_idx = geometries.sindex.intersection((point.bounds))
        embedding = preds[search_idx[0]]
        # compute the cosine similarity between the clicked point and all other points
        cos_sim = [1 - spatial.distance.cosine(embedding, pred) for pred in preds]
        # get the index of the point with the second highest cosine similarity
        idx = np.argsort(cos_sim)[-2]
        # create images for plotting
        search_image = np.clip((norm_chips[search_idx[0],:,:,2::-1] + 2) / 5, 0,1)
        #decoder_pred = decoder.predict(np.expand_dims(embedding, axis=0))[0]
        #decoder_img = np.clip((decoder_pred[:,:,2::-1] + 2) / 5, 0,1)
        match_img = np.clip((norm_chips[idx,:,:,2::-1] + 2) / 5, 0,1)
        # clear any existing figure from notebook cell and then plot images
        fig, ax = plt.subplots(1,3, figsize=(9,3), dpi=100)
        ax[0].imshow(search_image)
        ax[0].set_title('Search Image')
        ax[0].axis('off')
        ax[1].imshow(np.ones((1,1,3)))
        #ax[1].set_title('Reconstruction')
        ax[2].axis('off')
        ax[1].imshow(match_img)
        ax[1].set_title('Match Image')
        ax[1].axis('off')
        #plt.show()
        # get the centroid of the geometry with the highest cosine similarity
        centroid = geometries.geometry[idx].centroid.coords[0]
        # get the lat and lon of the centroid
        lat = centroid[1]
        lon = centroid[0]
        # set the map center to the centroid of the geometry with the highest cosine similarity
        m.center = (lat, lon)
# add a click event to the map
m.on_interaction(get_coords)
m

In [ ]:
encodings[0]